In [1]:
import os
import sys
os.path.dirname(sys.executable)

'C:\\ProgramData\\Anaconda3'

In [2]:
# !pip3 install -U selenium

In [3]:
# !pip3 install webdriver-manager

In [4]:
# !pip install beautifulsoup4 requests

In [3]:
import requests

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver
from selenium.webdriver.common.by import By
import pandas as pd

<!-- !pip install selenium -->

In [5]:
# # importing packages for managing web scrapping
# from selenium import webdriver
# # this is tested on Firefox or you can use "webdriver.Chrome()"
from bs4 import BeautifulSoup
import re
import time
from time import sleep

In [6]:
# import parsel

In [9]:
# import pandas as pd

In [10]:
# import getpass
# import csv
# from parsel import Selector

In [ ]:
#request user input for LinkedIn username and password:
print("Please enter the exact LinkedIn username you use to login (email/phone?):")
username_string = str(input()) 
print()
print("Please enter the exact LinkedIn password:")
password_string = str(input())
print()
print("Please enter your username exactly how it appears in your profile link (after '/in') :")
link_username = str(input())
print()
print("Please enter the number of the last posts you want to analyse:")
number_of_posts = int(input())

Please enter the exact LinkedIn username you use to login (email/phone?):


In [ ]:
# browser = webdriver.Chrome()

In [ ]:
# browser = webdriver.Chrome(executable_path="C:/Users/olaye/AppData/Roaming/Python/Python39/site-packages/chromedriver.exe.")

In [ ]:
# browser.get("https://www.google.com/")  # if you want to access google
# browser.get("https://www.linkedin.com/login")  # if you want to access linkedin

In [ ]:
# ?webdriver.Chrome

In [ ]:
# #open the LinkedIn login page and login under a specified account:
# browser.get('https://www.linkedin.com/login')

In [8]:
options = Options()
options.add_argument("start-maximized")
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
browser.get("https://www.linkedin.com/login")

[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.78M/6.78M [00:13<00:00, 539kB/s]


In [9]:
#enter the specified information to login to LinkedIn:
elementID = browser.find_element('id', 'username')

In [10]:
elementID.send_keys(username_string)

In [11]:
elementID = browser.find_element('id','password')

In [12]:
elementID.send_keys(password_string)

In [13]:
elementID.submit()

In [15]:
#open the recent post activity page of the LinkedIn user you specified:
recent_activity_link = "https://www.linkedin.com/in/" + link_username + "/detail/recent-activity/shares/"

In [16]:
browser.get(recent_activity_link)

In [26]:
type(browser)

selenium.webdriver.chrome.webdriver.WebDriver

In [25]:
element = browser.find_element("class","//input[@id='hue-web-icons-sprite']")
browser.execute_script("arguments[0].click();", element)

InvalidArgumentException: Message: invalid argument: invalid locator
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x003F37D3]
	(No symbol) [0x00388B81]
	(No symbol) [0x0028B36D]
	(No symbol) [0x002BD433]
	(No symbol) [0x002BD4BB]
	(No symbol) [0x002F3302]
	(No symbol) [0x002DB464]
	(No symbol) [0x002F1215]
	(No symbol) [0x002DB216]
	(No symbol) [0x002B0D97]
	(No symbol) [0x002B253D]
	GetHandleVerifier [0x0066ABF2+2510930]
	GetHandleVerifier [0x00698EC1+2700065]
	GetHandleVerifier [0x0069C86C+2714828]
	GetHandleVerifier [0x004A3480+645344]
	(No symbol) [0x00390FD2]
	(No symbol) [0x00396C68]
	(No symbol) [0x00396D4B]
	(No symbol) [0x003A0D6B]
	BaseThreadInitThunk [0x75436359+25]
	RtlGetAppContainerNamedObjectPath [0x77BF7C14+228]
	RtlGetAppContainerNamedObjectPath [0x77BF7BE4+180]


In [ ]:
########################################################
#______________STEP_2:_SCRAPE_POST_STATS_______________#
########################################################


#calculate number of scrolls depending on the input
number_of_scrolls = -(-number_of_posts // 5)  # 5 is LinkedIn's number of posts per scroll

#we need a loop because we have a particular number of scrolls...
likes = []
comments = []
views = []

SCROLL_PAUSE_TIME = 5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

for scroll in range(number_of_scrolls) : 
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height    

#query the contents (returns service reponse object with web contents, url headers, status and other):
src = browser.page_source
#beautiful soup instance:
soup = BeautifulSoup(src, features="lxml")   #lxml
    
#find LIKES on LinkedIn
#look for "span" tags that have the specific following attribute (click 'inspect' on the L-in page)
#need to convert the list of bs4 tags into strings and then extract 
#find these specific tags ("<stuff>") in the soup contents:

likes_bs4tags = soup.find_all("span", attrs = {"class" : "v-align-middle social-details-social-counts__reactions-count"})
#converts a list of 1 string to int, appends to likes list
for tag in likes_bs4tags:
    strtag = str(tag)
    #the first argument in findall (below) is a regular expression (accounts for commas in the number)
    list_of_matches = re.findall('[,0-9]+',strtag)
    #converts the last element (string) in the list to int, appends to likes list
    last_string = list_of_matches.pop()
    without_comma = last_string.replace(',','')
    likes_int = int(without_comma)
    likes.append(likes_int)
        
#find COMMENTS on LinkedIn
#same concept here
comments_bs4tags = soup.find_all("li", attrs = {"class" : "social-details-social-counts__item social-details-social-counts__comments"})
for tag in comments_bs4tags:
    strtag = str(tag)
    list_of_matches = re.findall('[,0-9]+',strtag)
    last_string = list_of_matches.pop()
    without_comma = last_string.replace(',','')
    comments_int = int(without_comma)
    comments.append(comments_int)
        
#find VIEWS on LinkedIn
#same concept here
views_bs4tags = soup.find_all("span", attrs = {"class" : "icon-and-text-container t-14 t-black--light t-normal"})
for tag in views_bs4tags:
    strtag = str(tag)
    list_of_matches = re.findall('[,0-9]+',strtag)
    last_string = list_of_matches.pop()
    without_comma = last_string.replace(',','')
    views_int = int(without_comma)
    views.append(views_int)  

print(likes) 
print(comments)
print(views)